## BANDIT IV

In [2]:
import numpy as np
from scipy.stats import t as t_law
import matplotlib.pyplot as plt
from tqdm import tqdm

colors = ['#377eb8', '#ff7f00', '#4daf4a','#f781bf', '#a65628', '#984ea3','#999999', '#e41a1c', '#dede00']
s_colors = ['#377eb8', '#ff7f00', '#4daf4a','#f781bf', '#a65628', '#984ea3','#999999', '#e41a1c', '#dede00']

In [21]:
#game parameters
n_iter = 50         #number of simulations
T = 1000             #time horizon
sigma_iv = 0.1         #standard deviation of endogenous noise
sigma_y = 0.1
rho = 1-2*np.random.binomial(1, 0.2, T) #random noise scaler for Y

k = 10      #number of arms
d = 5       #dimension of X

Gamma_star = np.random.randn(k,d)   # arms x features
Beta_star = np.random.randn(d)[...,None]      # features

#algorithm parameters
gamma_z = 0.1 
gamma_x = 0.1
G_t = [1 for _ in range(T)]
B_t = [1 for _ in range(T)]

assert k >= d #assumption

#intializing estimated parameter matrics
U_0 = np.eye(k)*gamma_z
V_0 = np.zeros((k,d))
W_0 = np.eye(d)*gamma_x
Q_0 = np.zeros((d,1))
 
#confidence sets #fill this out
C_1s = np.zeros(T)
C_2s = np.zeros(T)


    


In [26]:
for iteration in tqdm(range(n_iter)):

    # Construct X,Y,Z
    endo = np.random.randn(T, d)         #endogenous component of X. 
    eps = sigma_iv* np.random.randn(T, d)   #exogenous component source of X, shape = Time x n_feat
    Z = np.random.randn(T,k)                #instrumental variable. K arms. Related to X but exogenous wrt Y

    u = endo + eps                                  #subgausssian vector satisfying E[zu']=0
    X = (Gamma_star.T @ Z[...,None]).squeeze() + u  # shape T x d  (IV + exogenous component + endogenous component)
    
    v = sigma_y*np.random.randn(T)  
    Y_true = ((Gamma_star @ Beta_star).T @ Z[...,None]).squeeze() + v   #shape: T

    #intializing estimated parameter matrics
    U_0 = np.eye(k)*gamma_z         # k x k
    V_0 = np.zeros((k,d))           # k x d
    W_0 = np.eye(d)*gamma_x         # d x d
    Q_0 = np.zeros((d,1))           # d x 1
    
    #confidence sets #fill this out
    C_1s = []
    C_2s = []

    cumulative_regret = 0

    for t in range(1,T):
        z = Z[t-1][...,None]
        x = X[t-1][...,None]
        y_true = Y_true[t-1]

        U_t = U_0 + z@z.T 

        V_t = V_0 + z @ x.T                         

        Gamma_est = np.linalg.inv(U_t) @ V_t          #shape : d x k

        # add to set once so that it isnt empty?
        if len(C_1s)==0:
            C_1s.append(Gamma_est)
        # ||x||_A = sqrt(x'Ax) (index on columns) 
        gdiff = [(Gamma_star[:,i] - Gamma_est[:,i]) for i in range(Gamma_est.shape[1])]
        if all([np.sqrt(g.T @ U_t @ g) <= G_t[t] for g in gdiff]): 
            C_1s.append(Gamma_est)

        X_est = (z.squeeze() @ Gamma_est)[...,None]   #shape : d x 1

        W_t = W_0 + X_est @ X_est.T                   #operation is other way in paper but that yield 1x1 (not dxd)

        Q_t = Q_0 + X_est*y_true                      #Transpose on X is the other way in the paper : shape (k,1)

        Beta_est = np.linalg.inv(W_t) @ Q_t           #shape : k x 1


        if len(C_2s)==0:
            C_2s.append(Beta_est)
        b = (Beta_star - Beta_est)
        if np.sqrt(b.T @ W_t @ b) <= B_t[t]:
            C_2s.append(Beta_est)


        zt = np.argmax(
                    [np.max(
                        [np.max(
                            [((gamma.T @ z).squeeze() @ beta) for beta in C_2s]
                        ) for gamma in C_1s]
                    ) for z_ in Z]
                )
        #choose arm

        #Confused Here....

        X = X.squeeze()
        play = x[zt]

        X_temp = np.zeros((X.shape[0], X.shape[1]+1))
        print(X_temp.shape)
        X_temp[:X.shape[0], :X.shape[1]] = X
        print(x.shape)
        X_temp[-1:,] = x
        x = np.concat

        # print(play.shape)

        
        raise

        U_0, V_0, W_0, Q_0 = U_t, V_t, W_t, Q_t



        # cumulative_regret += y_true - y_pred

        # if t%10==0:
        #     print(cumulative_regret / t)
    


    

  0%|          | 0/50 [00:00<?, ?it/s]

(1000, 6)
(5, 1)


ValueError: could not broadcast input array from shape (5,1) into shape (1,6)

In [181]:
all([1,3,5]) 

True

## Scrap

In [165]:
for iteration in tqdm(range(n_iter)):

    # Construct X,Y,Z
    endo = np.random.randn(T, d)         #endogenous component of X. 
    eps = sigma_iv* np.random.randn(T, d)   #exogenous component source of X, shape = Time x n_feat
    Z = np.random.randn(T,k)                #instrumental variable. K arms. Related to X but exogenous wrt Y

    u = endo + eps                                  #subgausssian vector satisfying E[zu']=0
    X = (Gamma_star.T @ Z[...,None]).squeeze() + u  # shape T x d  (IV + exogenous component + endogenous component)
    
    v = sigma_y*np.random.randn(T)  
    Y_true = ((Gamma_star @ Beta_star).T @ Z[...,None]).squeeze() + v   #shape: T

    #intializing estimated parameter matrics
    U_0 = np.eye(k)*gamma_z         # k x k
    V_0 = np.zeros((k,d))           # k x d
    W_0 = np.eye(d)*gamma_x         # d x d
    Q_0 = np.zeros((d,1))           # d x 1
    
    #confidence sets #fill this out
    C_1s = []
    C_2s = []

    cumulative_regret = 0

    z = Z[0][...,None]
    x = X[0][...,None]
    y_true = Y_true[0]
    for t in range(1,T):

        U_t = U_0 + z@z.T 

        V_t = V_0 + z @ x.T                         

        Gamma_est = np.linalg.inv(U_t) @ V_t          #shape : d x k
        C_1s.append(Gamma_est)

        X_est = (z.squeeze() @ Gamma_est)[...,None]   #shape : d x 1

        W_t = W_0 + X_est @ X_est.T                   #operation is other way in paper but that yield 1x1 (not dxd)

        Q_t = Q_0 + X_est.T*y_true                    
        Beta_est = np.linalg.inv(W_t) @ Q_t           #shape : k x k

        C_2s.append(Beta_est)

        zt = np.argmax(
                    [np.max(
                        [np.max(
                            [((gamma.T @ z).squeeze() @ beta) for beta in C_2s]
                        ) for gamma in C_1s]
                    ) for z_ in Z]
                )
        #choose arm
        play = x[zt]
        x = 

        print(play.shape)





        

        
    

        U_0, V_0, W_0, Q_0 = U_t, V_t, W_t, Q_t



        # cumulative_regret += y_true - y_pred

        # if t%10==0:
        #     print(cumulative_regret / t)
    


    

  0%|          | 0/50 [00:00<?, ?it/s]

(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)


  0%|          | 0/50 [00:02<?, ?it/s]


KeyboardInterrupt: 

In [111]:
        az = []
        for z_ in Z: 
            z_ = Z
            ag = []
            for gamma in C_1s:
                ab = []
                for beta in C_2s:
                    ab.append((gamma.T @ z).squeeze() @ beta)
                m = np.max(ab)
                ag.append(m)
            m = np.max(ag)
            az.append(m)
        z = np.argmax(az)

        print(z)


(1000, 5, 1)

In [96]:


    for t in range(1,T):
        z = Z[t-1][...,None]
        x = X[t-1][...,None]
        y_true = Y_true[t-1]
        #problem here for k ≠ d (dim(U)=(k,k), dim(z_t @ z_t.T)=(d,d)
        #solution pad z with zeros
        z = pad_vector(z, k)
        U_t = U_0 + z@z.T 

        V_t = V_0 + z @ x.T                         

        Gamma_est = np.linalg.inv(U_t) @ V_t          #shape : d x k

        X_est = (z.squeeze() @ Gamma_est)[...,None]   #shape : d x 1

        W_t = W_0 + X_est @ X_est.T                   #operation is other way in paper but that yield 1x1 (not dxd)

        Q_t = Q_0 + X_est.T*y_true                    
        Beta_est = np.linalg.inv(W_t) @ Q_t           #shape : k x k
        U_0, V_0, W_0, Q_0 = U_t, V_t, W_t, Q_t

        y_pred = np.dot(Gamma_est, Beta_est)

        cumulative_regret = y_true - y_pred

    
    raise
    cumulative_regret /= T


    

  0%|          | 0/50 [00:00<?, ?it/s]

(5, 10)
(1000, 10)
(1000, 5)


ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 1000 is different from 10)

In [5]:
a = np.ones((3,1))
b = np.ones((3,1))

np.dot(a,b.T)

array([[1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.]])

In [16]:
X = np.ones((3,3))
X[1].uns.T @ X[2]

3.0

In [20]:
X[1].transpose()

array([1., 1., 1.])

In [ ]:
store_betas = np.zeros((len(methods), n_iters, d, T))

# Run
for it in tqdm(range(n_iters)):  # iterations to average the effect of noise
    
    # INITIALIZE
    thetas = np.zeros((len(methods),d,d))